# CI, CD & GitHub Actions

## Introduction
>GitHub Actions allow us to automate various stages of software development if GitHub is employed as the version-control system.

*__Key characteristics of GitHub actions__*

- __Event-driven:__ Within a GitHub repository, it is possible to automate the execution of a piece of code (e.g. pull request) based on event triggers.
- __Configuration-oriented:__ `.yaml` is employed to configure the necessary __workflows.__
- __Customisable:__ We can create custom GitHub Actions with `docker` or JavaScript.
- __Supports simple continuous integration (CI):__ Each pull request/merge can be automatically tested.
- __Supports simple continuous deployment/delivery (CD):__ After successful testing, we can create a pipeline that automatically deploys the code for others to use.


### Continuous integration

> __This refers to the integration of software changes from multiple contributors.__

Generally, this includes
- assessing the code quality (via automated code assessment, reviews or both).
- assessing adherence to the project's best practices.

All of the above and more can be achieved using GitHub Actions in order to streamline the code-merging process (e.g. assign labels automatically). 


### Continuous deployment

> __This refers to the testing of production code to ensure that it is always ready for deployment (i.e. bug-free).__

Generally, this includes
- inspecting changes against integration tests (or a whole suite of them).
- testing every feature that is to be merged.
- rapid deployment of necessary fixes (users always have the best possible version of the product available).

### Continuous delivery

> __This refers to the automatic deployment or code after testing and approval.__

This is often confused with continuous deployment.

__This approach may not be suitable for every project and may be difficult to achieve. However, it offers significant benefits if done correctly.__

Generally, one could set up, amongst others,
- automated/easy rollbacks (to catch critical flaws when tests fail),
- test optimisation to ensure that only the necessary tests are conducted to improve speed and reduce cost,
- automated deployment of the changes to production (e.g. creating images and containers, and deploying them on Kubernetes as a new version of the software).

## GitHub Actions Concepts

The key concepts in GitHub Actions include the following:

- __Events:__ Anything that occurs inside the repository, e.g. new issue, pull request, and merging (__external events are allowed via [webhooks](https://docs.github.com/en/rest/reference/repos#create-a-repository-dispatch-event)__).
- __Workflow:__ Automated processes added to the repository.
- __Jobs:__ Set of steps running on the same machine.
- __Steps:__ Individual tasks that can run __action__/shell commands on the machine.
- __Action:__ Standalone commands __set up individually in separate/the same GitHub repository.__
- __Runner:__ A server with [Github Actions runner](https://github.com/marketplace/actions/github-action-for-latex) installed.

![](./images/github-actions-design.png)

Things to note:
- The full list of events for triggering workflows can be found [here](https://docs.github.com/en/actions/reference/events-that-trigger-workflows).
- __Jobs in a workflow run in parallel by default.__
- __Dependencies can be created between jobs__ (e.g. deployment relies on the success of the testing job).
- __Any publicly set-up Action can be used__ (e.g. see the Action for creating LaTex documents [here](https://github.com/marketplace/actions/github-action-for-latex)).
- __Runners are provided by GitHub for free__ (see the limits for different tiers [here](https://docs.github.com/en/actions/reference/usage-limits-billing-and-administration)).
- It is also possible to set up custom runners.

## Structure

To use GitHub Actions, the following are required:
- an appropriate folder in our repository containing workflows (`/.github/workflows`).
- workflow `.yaml` files.

__Consider the example below:__

```yaml
name: learn-github-actions
on: [push]
jobs:
  check-bats-version:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2
      - uses: actions/setup-node@v1
      - run: npm install -g bats
      - run: bats -v

```

- `name`: specifies the name of the workflow
- `on`: event on which it will run
- `jobs`: list of jobs that will run in parallel, unless otherwise specified
- `check-bats-version`: our first job
    - `runs-on`: runs on the latest Ubuntu worker...
    - ... and contains the following `steps`:
        - `uses`: checks out our repository
        - `uses` `node`: for running `npm` commands
        - `run`s JS installation of bats package
        - `run`s the command that checks the `bats` version
        
Visually, it would appear similar to this:

![](images/github-actions-example-workflow-diagram.png)

## Possible Fields

There are numerous possible field configurations. For details, check [here](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions).

We go over some of the most useful fields.

### on

> Enables the specification of multiple events and their details based on which the workflow is run. 

Consider the following `.yaml` part:

```yaml

on:
  # Trigger the workflow on push or pull request,
  # but only for the main branch
  push:
    branches:
      - main
  pull_request:
  # Also trigger on page_build, as well as release created events
  page_build:
  release:
    types: [published, created, edited]
  schedule:
    # Since * is a special character in YAML, the string must be wrapped in quotes
    - cron:  '30 5,17 * * *'


```

There are a few possibilities for running the workflow based on predefined events:

- On `push`/`pull_request` branches and/or tags (see [here](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions#onpushpull_requestbranchestags)), including their regex exclusion/inclusion.
- On `push`/`pull_request` to a specific file/directory (see [here](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions#onpushpull_requestpaths)).

### env

> Allows us to set the availability of environment variables.

Availability can be set for different stages:
- all jobs
- single job
- single step

See the examples below:

```yaml
env:
  SERVER: production
jobs:
  job1:
    env:
      FIRST_NAME: Mona
    steps:
      - name: My first action
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
          FIRST_NAME: Mona
          LAST_NAME: Octocat
          
```

## Jobs Dependencies

> GitHub Actions allow us to specify the jobs that must achieve completion before the current one runs.

This approach enables the sequential running of jobs separately. 

For example:
- Test our application with multiple Python versions (using matrix).
- If all the tests are successful, move to the deployment stage.

By default, all required jobs must finish successfully before the job awaiting their completion runs.

See the example:

```yaml
jobs:
  job1:
  job2:
    needs: job1
  job3:
    needs: [job1, job2]
```

To ensure that the jobs finish, regardless of the result, do the following:

```yaml
jobs:
  job1:
  job2:
    needs: job1
  job3:
    if: always()
    needs: [job1, job2]
```

## Strategy and Matrix

> It is possible to specify a __strategy__ that specifies the matrix of the jobs to be run.

Using this approach, we can easily create many similar jobs via __variable substitution__.

Few things to note:
- The maximum number of jobs that can be generated this way is `256`.
- Ordering matters; the first defined option will be the first job.

Below, an example of a matrix containing `6` jobs is provided:

```yaml
runs-on: ${{ matrix.os }}
strategy:
  matrix:
    os: [ubuntu-18.04, ubuntu-20.04]
    node: [10, 12, 14]
steps:
  - uses: actions/setup-node@v2
    with:
      node-version: ${{ matrix.node }}
```

We can also add some specific configurations (e.g. a specific mix of OS and `node`, which is outside of the matrix) using `include` (or exclude with `exclude`):

```yaml
name: Node.js CI
on: [push]
jobs:
  build:
    runs-on: ${{ matrix.os }}
    strategy:
      matrix:
        os: [macos-latest, windows-latest, ubuntu-18.04]
        node: [8, 10, 12, 14]
        include:
          # includes a new variable of npm with a value of 6
          # for the matrix leg matching the os and version
          - os: windows-latest
            node: 8
            npm: 6
         exclude:
          # excludes node 8 on macOS
          - os: macos-latest
            node: 8
```

Additionally, there are a few important variables other than `matrix` that we can set to customise the behaviour:
- `strategy.fail-fast`: either `true` or `false`; cancels the whole job if `true` (default: `true`).
- `strategy.max-parallel`: the number of parallel jobs in the matrix. By default, as many as possible (depending on the number of cores).
- The variable for continuing if one job fails, see below:

```yaml
runs-on: ${{ matrix.os }}
continue-on-error: ${{ matrix.experimental }}
strategy:
  fail-fast: false
  matrix:
    node: [13, 14]
    os: [macos-latest, ubuntu-18.04]
    experimental: [false]
    include:
      - node: 15
        os: ubuntu-18.04
        experimental: true
```

## Step-Specific Arguments

> Step is the essential unit that defines the role of the workflow; thus, it offers a few additional fields.

Each step
- can (but does not have to) run an action (note that all actions run as steps).
- has access to the system defined in the `runs-on` clause.
- runs a separate process.

Additionally, the number of steps is __unlimited__.

Consider the example below:

```yaml
name: Greeting from Mona

on: [push, pull_request]

jobs:
  my-job:
    name: My Job
    runs-on: ubuntu-latest
    steps:
      - name: Print a greeting
        env:
          MY_VAR: Hi there! My name is
          FIRST_NAME: Mona
          MIDDLE_NAME: The
          LAST_NAME: Octocat
        run: |
          echo $MY_VAR $FIRST_NAME $MIDDLE_NAME $LAST_NAME.
      - name: My first step
        if: ${{ github.event_name == 'pull_request' && github.event.action == 'unassigned' }}
        run: echo "This event is a pull request and has no assignees".
```

### The uses and with fields

> These are employed to select an action to run as part of a step in your job (an action is a standalone piece of code that performs a specific task, e.g. sets up a specific version of Python).

*Usage Tips*
- Specify the version directly (similar to the case with Docker tags).
- Specify the major version for fixes.

For example,

```yaml
steps:    
  # Reference a specific commit
  - uses: actions/setup-node@c46424eee26de4078d34105d3de3cc4992202b1e
  # Reference the major version of a release
  - uses: actions/setup-node@v1
  # Reference a minor version of a release
  - uses: actions/setup-node@v1.2
  # Reference a branch
  - uses: actions/setup-node@main
```

To explore the possibilities, check [here](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions#example-using-a-public-action).

Additionally, it is worth noting that the `uses` field often comes with the `with` nested field, which allows us to specify arguments for the action.

The `workflow` below specifies the checkout from the private repository (__Please pay attention to the arguments)__:

```yaml
jobs:
  my_first_job:
    steps:
      - name: Check out repository
        uses: actions/checkout@v2
        with:
          repository: octocat/my-private-repo
          ref: v1.0
          token: ${{ secrets.PERSONAL_ACCESS_TOKEN }}
          path: ./.github/actions/my-private-repo
      - name: Run my action
        uses: ./.github/actions/my-private-repo/my-action
```

Here is another simple example:

```yaml
jobs:
  my_first_job:
    steps:
      - name: My first step
        uses: actions/hello_world@main
        with:
          first_name: Mona
          middle_name: The
          last_name: Octocat 
```

> __Please refer to the individual actions documentation for more information on the mandatory/optional arguments.__

### The run field

Another option is to use `run` instead of `action`.

> The __run field specifies the shell command that will run on the OS.__

Simply pass the `shell` command into the field, and optionally specify the working directory with respect to the repository's root:

```yaml
steps:
  - name: Clean temp directory
    run: rm -rf *
    working-directory: ./temp
```

You could also specify a different shell to run the command (`bash` is the default shell), e.g.

```yaml
steps:
  - name: Display the path
    run: echo $PATH
    shell: bash
```

> __Python is available as an optional shell; therefore, it is possible to run Python commands directly.__

## Reading Exercise

Do a line-by-line read-through of the example `test + deployment` workflow below.

- __section `name` to `jobs`__
- __`jobs.tests` - `jobs.tests.steps`__
- __`jobs.tests.steps` - `jobs.docker`__
- __`jobs.docker` - `jobs.docker.steps`__
- __`jobs.docker.steps` - `jobs.pip`__
- __`jobs.pip` - until the end__


```yaml
---
name: update
on:
  push:

jobs:
  tests:
    name: ${{ matrix.os }}-py${{ matrix.python }}-torch${{ matrix.pytorch }}
    runs-on: ${{ matrix.os }}
    strategy:
      matrix:
        pytorch:
          - "v1.6.0"
          - "latest"
        python:
          - 3.6
          - 3.7
          - 3.8
        os:
          - ubuntu-latest
          - ubuntu-16.04
    steps:
      - uses: actions/checkout@v1
      - name: Set up Python ${{ matrix.python }}
        uses: actions/setup-python@v1
        with:
          python-version: ${{ matrix.python }}
      - name: Update torchlambda version
        run: ./scripts/release/update_version.sh
      - name: Install dependencies
        run: ./scripts/ci/dependencies.sh
      - name: Build docker image locally
        run: ./scripts/ci/build.sh ${{ matrix.pytorch }}
      - name: Perform tests
        run: ./tests/settings/automated/run.sh ${{ matrix.pytorch }}
      - name: Upload test results
        uses: actions/upload-artifact@v1
        with:
          name: ${{ matrix.os }}-py${{ matrix.python }}-torch${{ matrix.pytorch }}.npz
          path: analysis.npz

  docker:
    needs: tests
    name: Deployment image szymonmaszke/torchlambda:${{ matrix.pytorch}}
    runs-on: ubuntu-latest
    strategy:
      matrix:
        pytorch:
          - "v1.6.0"
          - "latest"
    steps:
      - uses: actions/checkout@v1
      - name: Set up Python
        uses: actions/setup-python@v1
        with:
          python-version: 3.7
      - name: Update torchlambda version
        run: ./scripts/release/update_version.sh
      - name: Install dependencies
        run: ./scripts/ci/dependencies.sh
      - name: Build docker image locally
        run: ./scripts/ci/build.sh ${{ matrix.pytorch }}
      - name: Login to Docker
        run: >
          docker login
          -u ${{ secrets.DOCKER_USERNAME }}
          -p ${{ secrets.DOCKER_PASSWORD }}
      - name: Deploy image szymonmaszke/torchlambda:${{ matrix.pytorch }}
        run: >
          docker push szymonmaszke/torchlambda:${{ matrix.pytorch }}

  pip:
    needs: tests
    name: Create and publish package to PyPI with current timestamp
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@master
      - name: Update torchlambda version
        run: ./scripts/release/update_version.sh
      - name: Set up Python
        uses: actions/setup-python@v1
        with:
          python-version: "3.7"
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install setuptools wheel
      - name: Build package
        run: python setup.py sdist bdist_wheel
      - name: Publish package to PyPI
        uses: pypa/gh-action-pypi-publish@master
        with:
          password: ${{ secrets.PYPI_PASSWORD }}
```

## Contexts and Expressions

> Contexts are a set of defined (__or predefined__) variables used within workflows.

They can be accessed using the __expression syntax__ and __index__ access.
 
For example,

```yaml
name: CI
on: push
jobs:
  prod-check:
    # Expression syntax; GitHub is the context, and ref is the key.
    # We could use github.ref instead
    if: ${{ github['ref'] == 'refs/heads/main' }}
    runs-on: ubuntu-latest
    steps:
      - run: echo "Deploying to production server on branch $GITHUB_REF"

```

Here are a few commonly used contexts:

- `github`: [docs](https://docs.github.com/en/actions/reference/context-and-expression-syntax-for-github-actions#github-context); GitHub-related info and info about the current run.
- `env`: [docs](https://docs.github.com/en/actions/reference/context-and-expression-syntax-for-github-actions#env-context); environment variables created within the workflow using `env`.
- `job`: [docs](https://docs.github.com/en/actions/reference/context-and-expression-syntax-for-github-actions#job-context); variables related to the current job.
- `steps`: [docs](https://docs.github.com/en/actions/reference/context-and-expression-syntax-for-github-actions#steps-context); variables related to the step(s).
- `secrets`: [docs](https://docs.github.com/en/actions/reference/encrypted-secrets); allow the specification of secrets, such as passwords, in the repository.__

Specifically, secrets can be specified in `settings`, as shown in the screenshot below:

![](images/github_workflow_secrets_setting_up.png)

- __Note that the secrets specified in `actions` will be available in the context for others to use.__
- Additionally, these secrets will be accessible repository-wide, but will not be encrypted.

See the example below:

```yaml
steps:
  - name: Hello world action
    with: # Set the secret as an input
      super_secret: ${{ secrets.SuperSecret }}
    env: # Or as an environment variable
      super_secret: ${{ secrets.SuperSecret }}
```

> __Most of the data required for custom actions and/or our scripts have already been provided by GitHub (e.g. branch name, event name, runner name, etc.).__


### Literals

> When using expressions, we can employ a set of values for comparison.

Consider the values below:


```yaml
env:
  myNull: ${{ null }}
  myBoolean: ${{ false }}
  myIntegerNumber: ${{ 711 }}
  myFloatNumber: ${{ -9.2 }}
  myHexNumber: ${{ 0xff }}
  myExponentialNumber: ${{ -2.99-e2 }}
  myString: ${{ 'Mona the Octocat' }}
  myEscapedString: ${{ 'It''s open source!' }}
```

The comparison operators are also standard:

| Operator  | Description |
| ------------- | ------------- |
| ()| Logical grouping |
| []| Index |
| .| Property dereference |
| !| Not |
| <| Less than |
| <=| Less than or equal |
| >| Greater than |
| >=| Greater than or equal |
| ==| Equal |
| !=| Not equal |
| &&| And |
| \|\| | Or |

__Things to note__
- GitHub casts data to numerical types if they do not match.
- Comparison is case insensitive.
- Objects are equated based on instance, not data.__

## Functions

Besides predefined environment variables, configurable context and others, GitHub Actions provide a few functions to use within the `.yaml`.

Here are a few examples:
- `contains(iterable, item)`: `true` if the `iterable` contains an `item`, e.g. `contains(github.event.issue.labels.*.name, 'bug')`.
- `{starts, ends}With(string, string)`: for example, `startsWith('Hello world', 'He') `.
- `format(string, r1, r2, ..., rN)`: similar to the case in Python, e.g. `format("Current reference on branch: {0}", github['ref'])`.

> __Note: In `github.event.issue.labels.*.name`, any matching key in place of `*` will be returned. For example, `[bug, first-issue, fix]`.__

- `{to, from}JSON(value)`: returns the JSON pretty-printed value of `value`, e.g. `toJSON(job)` might return `{ "status": "Success" }`.

These functions can be exploited to pass data between jobs, as follows:

```yaml
name: build
on: push
jobs:
  job1:
    runs-on: ubuntu-latest
    outputs:
      matrix: ${{ steps.set-matrix.outputs.matrix }}
    steps:
      - id: set-matrix
        run: echo "::set-output name=matrix::{\"include\":[{\"project\":\"foo\",\"config\":\"Debug\"},{\"project\":\"bar\",\"config\":\"Release\"}]}"
  job2:
    needs: job1
    runs-on: ubuntu-latest
    strategy:
      matrix: ${{fromJSON(needs.job1.outputs.matrix)}}
    steps:
      - run: build
```


### Status-check functions

> These allow us to determine the status of previous jobs and to branch with the `if` conditional based on the status.

Consider the most common status-check function, `success()`:

```yaml
steps:
  ...
  - name: The job has succeeded
    if: ${{ success() }}
```

Others include
- `failure()`
- `cancelled()` (manually)
- `always()` (is always `true`, regardless of the previous status)

## Guides

GitHub Actions provide many guides for common use cases.

They can be found [here](https://docs.github.com/en/actions/guides), and they contain, among other things, information on how to
- build your Python code and test it.
- deploy to Kubernetes.
- upload your packages to `pypi`.
- manage open issues.
- manage pull requests

## Conclusion
At this point, you should have a good understanding of
- GitHub Actions.
- how to automate various stages of software development using GitHub Actions.
- key GitHub Actions concepts. 